In [ ]:
!pip3 install seaborn
!pip3 install geopandas

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = pd.read_csv('../data/curated/processed.csv')

In [ ]:
CORR_COLS = ["passenger_count", "trip_distance", "fare_amount", "extra",
             "mta_tax", "tip_amount", "total_amount", "week", "trip_time", "weekday", "starting_hour","ending_hour","PULocationID","DOLocationID"]
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
sns.heatmap(data[CORR_COLS].corr(),ax=axs)

In [ ]:
# plot the Counts to Whether the Passenger Give Tips
cleaned[cleaned_data['tip_amount']==0].count()

In [ ]:
WEATHER_COL = ["tip_amount","feelslike","humidity","windspeed","visibility","solarradiation","conditions_Clear","conditions_Partially cloudy","conditions_Rain, Overcast","conditions_Rain, Partially cloudy","conditions_Snow, Rain, Overcast","conditions_Snow, Rain, Partially cloudy"]
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
sns.heatmap(data[WEATHER_COL].corr(),ax=axs)

In [ ]:
weather_feature = ['windspeed','visibility','temp','humidity']
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(15,10))
axs = [ax1, ax2, ax3, ax4]
for feature, ax in zip(weather_feature, axs):
    sns.distplot(weather_data[feature], ax=ax, bins=20)
#plt.suptitle('Weather Feature Distribution',fontsize=30,y=0.95)
plt.show()


In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize=(15,10))
axs = [ax1, ax2, ax3, ax4]
for feature, ax in zip(weather_feature, axs):
    data_log = np.log(weather_data[feature]+0.01)
    sns.distplot(data_log, ax=ax, bins=20)
#plt.suptitle('Weather Feature Log Distribution',fontsize=30,y=0.95)
plt.show()

In [ ]:
taxi_feature = ['fare_amount','tip_amount']
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(8,2))
axs = [ax1, ax2, ax3, ax4]
for feature, ax in zip(taxi_feature, axs):
    sns.distplot(cleaned[feature], ax=ax, bins=20)
#plt.suptitle('Taxi Fare Distribution',fontsize=30,y=0.95)
plt.show()

In [ ]:
# trip freq by hour
plt.figure(figsize=(9, 6))
count = data.groupby("starting_hour").count()["tpep_pickup_datetime"]
plt.plot(count.index, count)
plt.ylabel("Time")
plt.xlabel("Hours")
plt.title('Frequency of Trip in each Hour')
plt.savefig('Frequency of Trip in each Hour.png')
plt.show()

In [ ]:
# trip freq by week day
plt.figure(figsize=(9, 6))
count = data.groupby("weekday").count()["tpep_pickup_datetime"]
index = ['Sun', 'Mon', 'Tue', 'Wed','Thu','Fri','Sat']
plt.plot(index, count)
plt.ylabel("Time")
plt.xlabel("Day")
plt.title('Frequency of Trip in each Day in week')
plt.savefig('Frequency of Trip in each Day in week.png')
plt.show()

In [ ]:
# average trip by weather
plt.figure(figsize=(9, 6))
ave_trip = data.groupby("conditions").count()["tpep_pickup_datetime"]
#plt.title('Average tip by weather conditions')
sns.barplot(ave_trip.index, ave_trip)

In [ ]:
# average tip by weather
plt.figure(figsize=(9, 6))
ave_tip = data.groupby("conditions").sum()['tip_amount']/data.groupby("conditions").count()["tpep_pickup_datetime"]
#plt.title('Average tip by weather conditions')
sns.barplot(ave_tip.index, ave_tip)
plt.xticks(rotation=45)

In [ ]:
# average tip by hour
plt.figure(figsize=(9, 6))
ave_tip = data.groupby("starting_hour").sum()['tip_amount']/data.groupby("starting_hour").count()["tpep_pickup_datetime"]
plt.title('Average tip by hour')
sns.barplot(ave_tip.index, ave_tip)

In [ ]:
# average tip by hour
plt.figure(figsize=(9, 6))
ave_tip = data.groupby("weekday").sum()['tip_amount']/data.groupby("weekday").count()["tpep_pickup_datetime"]
plt.title('Average tip by weekday')
sns.barplot(['Sun', 'Mon', 'Tue', 'Wed','Thu','Fri','Sat'], ave_tip)

In [ ]:
import geopandas as gpd

# sf stands for shape file
sf = gpd.read_file("../data/raw/taxi_zones.shp")
zone = pd.read_csv("../data/raw/taxi+_zone_lookup.csv")

# Convert the geometry shaape to to latitude and longitude
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

In [ ]:
DO_count = data['DOLocationID'].value_counts().sort_index()
DO_count = pd.DataFrame({"LocationID":DO_count.index, "DO_Count":DO_count})
gdf = gpd.GeoDataFrame(pd.merge(DO_count, sf, left_on='LocationID', right_on='LocationID')).drop('LocationID', axis=1)
gdf.sort_values("DO_Count",ascending=False)

In [ ]:
# plot the map showing drop off locations
variable = 'DO_Count'
vmin, vmax = 0, gdf['DO_Count'].max()
fig, ax = plt.subplots(1, figsize=(11,11))
ax.set_title('Average Dropoff in each Region')
sn = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sn.set_array([])
fig.colorbar(sn)
gdf.plot(column=variable, cmap='Blues', linewidth=1.5, ax=ax, edgecolor='0.4')
plt.savefig('Average Dropoff in each Region.png')
plt.show()

In [ ]:
fare_region = data.groupby('PULocationID')['fare_amount'].sum()/data['PULocationID'].value_counts().sort_index()
fare_region = pd.DataFrame({"LocationID":fare_region.index, "fare_region":fare_region})
gdf = gpd.GeoDataFrame(pd.merge(fare_region, sf, left_on='LocationID', right_on='LocationID')).drop('LocationID', axis=1)
top_10 = gdf.sort_values("fare_region",ascending=False)[:10]
top_10

In [ ]:
# plot the map showing ave tip in each region
variable = 'fare_region'
vmin, vmax = 0, gdf['fare_region'].max()
fig, ax = plt.subplots(1, figsize=(11,11))
ax.set_title('Average Fare per Trip in each Region')
sn = plt.cm.ScalarMappable(cmap='Oranges', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sn.set_array([])
fig.colorbar(sn)
gdf.plot(column=variable, cmap='Oranges', linewidth=1.5, ax=ax, edgecolor='0.4')
plt.savefig('Average Fare per Trip in each Region.png')
plt.show()

In [ ]:
tip_region = data.groupby('PULocationID')['tip_amount'].sum()/data['PULocationID'].value_counts().sort_index()
tip_region = pd.DataFrame({"LocationID":tip_region.index, "tip_region":tip_region})
gdf = gpd.GeoDataFrame(pd.merge(tip_region, sf, left_on='LocationID', right_on='LocationID')).drop('LocationID', axis=1)
top_10 = gdf.sort_values("tip_region",ascending=False)[:10]
top_10

In [ ]:
# plot the map showing ave tip in each region
variable = 'tip_region'
vmin, vmax = 0, gdf['tip_region'].max()
fig, ax = plt.subplots(1, figsize=(11,11))
ax.set_title('Average Tip per Trip in each Region')
sn = plt.cm.ScalarMappable(cmap='Greens', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sn.set_array([])
fig.colorbar(sn)
gdf.plot(column=variable, cmap='Greens', linewidth=1.5, ax=ax, edgecolor='0.4')
plt.savefig('Average Tip per Trip in each Region.png')
plt.show()